# Test viscous liquid transfer

The objective of this Jupyter notebook is to guide the user through the process of obtaining the liquid handling parameters that can accutately transfer volumes of viscous liquids using a rLine1000 automatic pipette. The process is based on a gravimetric method where the volume transfered with the autoamted pipette is compared with the expected mass for the transfer. The parameters are tuned to minimize the relative error calculated from the measured mass in comparison to the expected mass for each transfer. 


## Initialization of automated plarform

In [1]:
#Import relevant python packages
import pandas as pd
import time


from pathlib import Path
import sys
REPOS = 'GitHub'
ROOT = str(Path().absolute()).split(REPOS)[0]
sys.path.append(f'{ROOT}{REPOS}')

from polylectric.configs.SynthesisB1 import SETUP, LAYOUT_FILE

from controllably import load_deck      # optional
load_deck(SETUP.setup, LAYOUT_FILE)     # optional

platform = SETUP
platform.mover.verbose = False


Import: OK <controllably.misc.decorators>
Import: OK <controllably.misc.helper>
Import: OK <controllably.misc.factory>
Import: OK <controllably.misc.layout>
Import: OK <controllably.misc.logger>
Import: OK <controllably.misc.misc_utils>


Import: OK <controllably.Compound.compound_utils>
Import: OK <controllably.Compound.LiquidMover.liquidmover_utils>


Import: OK <controllably.Move.move_utils>
Import: OK <controllably.Move.Jointed.jointed_utils>
Import: OK <controllably.Move.Jointed.Dobot.dobot_utils>
Import: OK <controllably.Move.Jointed.Dobot.m1pro_utils>
Import: OK <controllably.Move.Jointed.Dobot.mg400_utils>
ClearError()
receive: ClearError()
EnableRobot()
receive: EnableRobot()
User(0)
receive: User(0)
Tool(0)
receive: Tool(0)
SetArmOrientation(1,1,1,1)
receive: SetArmOrientation(1,1,1,1)
Infeasible coordinates! (0.0, 0.0, 200.0)
Infeasible coordinates! (300.0, 1.1368683772161603e-13, 0.0)
MovJ(300.000000,0.000000,200.000000,10.000000,0.000000,0.000000)
Move time: 3.000s (1.000x

In [2]:
#Initialization of variables for platfomr objects
pipette= platform.setup
deck = platform.setup.deck
balance = platform.balance
balance_deck = deck.slots['1']
source = deck.slots['2']
tip_rack = deck.slots['3']
bin = deck.slots['4']


In [ ]:
#Stablish initial height of liquid on the source vial
liquid_level = 17

In [3]:
#Initialize OT2 for pipetting

pipette.attachTip()

Tip capacitance: 270
SpeedFactor(20)
receive: SpeedFactor(20)
MovJ(300.000000,0.000000,200.000000,10.000000,0.000000,0.000000)
Move time: 0.000s (0.200x)
[3.00000000e+02 1.13686838e-13 2.00000000e+02], [10  0  0]
SpeedFactor(100)
receive: SpeedFactor(100)
MovJ(198.894071,-3.942538,200.000000,10.000000,0.000000,0.000000)
Move time: 1.011s (1.000x)
[198.89407123  -3.94253829 200.        ], [10  0  0]
SpeedFactor(20)
receive: SpeedFactor(20)
MovJ(198.894071,-3.942538,72.400000,10.000000,0.000000,0.000000)
Move time: 6.380s (0.200x)
[198.89407123  -3.94253829  72.4       ], [10  0  0]
SpeedFactor(100)
receive: SpeedFactor(100)
SpeedFactor(1)
receive: SpeedFactor(1)
MovJ(198.894071,-3.942538,52.400000,10.000000,0.000000,0.000000)
Move time: 20.000s (0.010x)
[198.89407123  -3.94253829  52.4       ], [10  0  0]
SpeedFactor(100)
receive: SpeedFactor(100)
SpeedFactor(20)
receive: SpeedFactor(20)
MovJ(198.894071,-3.942538,169.300000,10.000000,0.000000,0.000000)
Move time: 5.845s (0.200x)
[198.89

array([325.1 ,  80.65, 105.9 ])

In [ ]:
pipette.aspirateAt()

## Viscous liquid protocol: Coarse approximation of pipetting parameters

The first step is to obtain approximate values of aspiration and dispense rates that can be used to initia;ize the ;iqiid transfer such as aspiration and dispense rates. 

In [ ]:
#This commands will aspirate 1000ul liquid at standard flow_rate.aspirate of pippette. A timer well be started just before aspiration starts
pipette.mover.safeMoveTo(source.wells['A1'].from_bottom((0,0,liquid_level-15)))
start = time.time()
pipette.liquid.aspirate(1000, speed = 265)

In [ ]:
#Run this cell when no further flow of liquid into the pipette tip is observed. Calculates an approximate flow rate for 
#aspiration
finish = time.time()
t_aspirate = finish-start
flow_rate_aspirate = 1000/t_aspirate
flow_rate_aspirate

In [ ]:
#This commands will dispense 900 ul of liquid at a flow rate equal to the approximation for aspiration obtained above.
#User should observe the behaviour of the dispense to make an educated guess of initial dispense rate to be implemented.
pipette.mover.safeMoveTo(source.wells['A1'].top)
pipette.liquid.aspirate(1000, rate = flow_rate_aspirate)


In [ ]:
liquid_level = liquid_level - 2*1000/1000

In [ ]:
#This command will clear out remaining liquid in the tip if the dispense was incomplete.
# pipettes['p1000'].home_plunger()
# protocol.delay(seconds=10)
# pipettes['p1000'].blow_out(location = source.wells_by_name()['C4'].top())
# pipettes['p1000'].touch_tip(location = source.wells_by_name()['C4'])

# pipettes['p1000'].home_plunger()
# protocol.delay(seconds=10)
# pipettes['p1000'].blow_out(location = source.wells_by_name()['C4'].top())
# pipettes['p1000'].touch_tip(location = source.wells_by_name()['C4'])

# pipettes['p1000'].home_plunger()
# protocol.delay(seconds=10)
# pipettes['p1000'].blow_out(location = source.wells_by_name()['C4'].top())
# pipettes['p1000'].touch_tip(location = source.wells_by_name()['C4'])
# pipettes['p1000'].move_to(source.wells_by_name()['C4'].top())


## Viscous liquid protocol: Gravimetric analysis for obtaining pipetting parameters to transfer a viscous liquid 
The following cells contain the code required to implemenet the gravimetric analysis of volume transfer of a specific viscousl iquid. User only needs to input the targetr volume, density of the liquid and mass of vials before and after a dispense.

First initialize or load a previous dataframe to record transfer parameters. Use only one of the next to cells. 

In [ ]:
#New dataframe
df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate','blow_out', 'delay_aspirate', 'delay_dispense', 'delay_blow_out', 'density', 'time', 'm', '%error', 'Transfer_Observation', 'Comment'])
df = df.astype({'liquid':str,'pipette':str,'blow_out':bool,'Transfer_Observation':str,'Comment':str})

In [ ]:
#Load dataframe
liquid_name = "Viscosity_std_9884"
density = 0.8844
pipette_name = 'rLine'


Define liquid to be handeled and input initial aspiration and dispense rates

In [ ]:
liquids_dict = {
  "Viscosity_std_9884" :{
        "rLine1000": {
            "aspiration_rate": 2 , 
            "dispense_rate": 0.5, 
            "blow_out" : False, 
            "delay_aspirate" : 3, 
            "delay_dispense" : 0, 
            "delay_blow_out" : 3, 
            },
    }

}


Transfer viscous liquds, input pippette name (pipette), desired volume (volume) to be dispensed in ul, liquid dictonary key string (liquid), density (density) and initial vial mass (mi). The code will register the liquid handling parameters used into the dataframe  

In [ ]:
volume=1000

pipette.mover.setSpeed(50)

if pipette.liquid.isTipOn()== False:
    pipette.attachTip()

start = time.time()

pipette.aspirateAt(volume= volume, coordinates= source.wells['A1'].from_bottom((0,0,liquid_level)), speed = liquids_dict[liquid_name][pipette_name]['aspiration_rate'])
time.sleep(liquids_dict[liquid_name][pipette_name]['delay_aspirate'])

pipette.mover.moveTo(source.wells['A1'].top)
pipette.mover.setSpeed(25)
pipette.mover.moveTo(source.wells['A1'].from_top((17,0,0)))
pipette.mover.moveTo(source.wells['A1'].from_top((-17,0,0)))
pipette.mover.moveTo(source.wells['A1'].from_top((0,17,0)))
pipette.mover.moveTo(source.wells['A1'].from_top((0,-17,0)))
pipette.mover.setSpeed(50)

# pipette.touch_tip()
balance.zero()
balance.clearCache()
balance.toggleRecord(True)
time.sleep(10)


pipette.dispenseAt(volume, balance_deck.wells['A1'].from_top((0,0,-5)), rate = liquids_dict[liquid_name][pipette_name]['dispense_rate'])
time.sleep(liquids_dict[liquid_name][pipette_name]['delay_dispense'])


if liquids_dict[liquid_name][pipette_name]['blow_out_rate'] == True:
    pipette.liquid.blowout()
    time.sleep.delay(seconds = liquids_dict[liquid_name][pipette_name]['delay_blow_out'])

finish = time.time()
time_m = finish - start

pipette.mover.safeMoveTo(source.wells['A1'].top)
time.sleep(10)
balance.toggleRecord(False)


df = df.append(liquids_dict[liquid_name][pipette], ignore_index = True)

Input mass of vial after transfer (mf). Code will calculate the relative error of transfer

In [ ]:
m = balance.buffer_df.loc[-10:,'Mass'].mean()
error = (m-density*volume/1000)/(density/1000*volume)*100
df.iloc[-1,-7] = time_m
df.iloc[-1,2] = volume
df.iloc[-1, 0] = liquid_name
df.iloc[-1, 1] = pipette_name
df.iloc[-1,-8] = density
df.iloc[-1, -4] = m
df.iloc[-1,-3]= error


In [ ]:
#Update liquid level
liquid_level = liquid_level - 2*volume/1000

In [ ]:
#Observe error made
df

In [ ]:
#Assign category of observation of transfer such as Incomplete Dispense, Incomplete Aspiration, 
#Incomplete Aspiration and Dispense, Complete Transfer. 
#Comment if any unexpected exprimental mistakes or changes were performed that have to be taken into account.
df.iloc[-1,-2]= 'Incomplete Dispense'
df.iloc[-1,-1]= 'No comment'

Save data, can be used at any time.

In [ ]:
df.to_csv(liquid_name+'.csv', index = False)

## Auxiliary code